### 15-2 위험 중립 할인

**15-2-1 날짜 모형과 조작**

In [1]:
import datetime as dt

In [2]:
dates = [dt.datetime(2015,1,1), dt.datetime(2015,7,1),dt.datetime(2016,1,1)]

In [3]:
#1년을 기준으로 한 비율 숫자로 표현하는 방법
(dates[1]-dates[0]).days/365. , (dates[2]-dates[1]).days/365

(0.4958904109589041, 0.5041095890410959)

In [4]:
fractions = [0.0,0.5,1.0]

In [5]:
#위 표현 같지만 완벽하게 같지 않음. 그 이유는 오전0시에 맞추기 힘들어서 > 연비율로 변환할 필요 있음

In [6]:
#
# DX Library Frame
# get_year_deltas.py
#
import numpy as np

def get_year_deltas(date_list, day_count= 365.):
    '''
    날짜를 1년 비율로 표현한 부동소수점 벡터 반환, 초기값 = 0
    
    argument
    ====
    date_list : list or array
        datetime 객체모음
    day_count : float
        1년의 날짜 수 
    
    return value
    ====
    delta_list : array
        1년 기준 비율
    '''
    start = date_list[0]
    delta_list = [(date-start).days/day_count for date in date_list]
    
    return np.array(delta_list)

In [7]:
dates = [dt.datetime(2015,1,1), dt.datetime(2015,7,1),dt.datetime(2016,1,1)]
get_year_deltas(dates)
#단기 이자율 모형에서 유용

array([0.        , 0.49589041, 1.        ])

**15-2-2 고정 단기 이자율**

In [9]:
#
# DX Libarary Frame
# constant_short_rate.py
#
#from get_year_deltas import <-만약 get_year_delta가 .py형태로 있으면 #풀어주기

class constant_short_rate(object):
    '''
    고정 단기 이자율 할인 클래스
    
    속성
    ----
    name : string
        객체 이름
    short_rate : float(positive)
        할인에 사용될 고정 이자율
    
    method
    ----
    get_discount_factors :
        datetime 1년 기준 비율 주어지면 할인율 반환
    '''
    def __init__(self,name,short_rate):
        self.name = name
        self.short_rate = short_rate
        if short_rate < 0 :
            raise ValueError('Short rate negative')
    def get_discount_factors(self,date_list,dtobjects = True):
        if dtobjects is True : 
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate*np.sort(-dlist))
        return np.array((date_list,dflist)).T

In [10]:
dates = [dt.datetime(2015,1,1), dt.datetime(2015,7,1),dt.datetime(2016,1,1)]
csr = constant_short_rate('csr',0.05)

In [11]:
csr.get_discount_factors(dates)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.951229424500714],
       [datetime.datetime(2015, 7, 1, 0, 0), 0.9755103387657228],
       [datetime.datetime(2016, 1, 1, 0, 0), 1.0]], dtype=object)

In [12]:
#1년 기준 비율도 출력 가능 (1년 기준 비율, 할인율 대응)
deltas = get_year_deltas(dates)
csr.get_discount_factors(deltas,dtobjects=False)

array([[0.        , 0.95122942],
       [0.49589041, 0.97551034],
       [1.        , 1.        ]])

### 15-3 시장 환경
사전 객체 사용법(4장 참조)

In [14]:
#
# DX Libarary Frame
# market_environment.py
#
class market_environment(object):
    '''
    가치평가와 관련된 시장 환경 모형을 위한 class
    
    속성
    ---
    name : string 
        시장 환경 이름
    pricing_date : datetime object
        시장 환경 날짜
        
    method
    ---
    add_constant :
        상수 추가(모형 파라미터)
    get_constant : 
        상수 반환
    add_list :
        (기초자산 등) 리스트 추가
    get_list :
        list 반환
    add_curve :
        시장 커브 추가 ( ex 이자율 커브)
    get_curve :
        시장 커브 반환
    add_environment :
        상수, list , curve 등 전체 시장 환경 추가 or 갱신
    '''
    
    def __init__(self,name,pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
    
    def add_constant(self, key, constant):
        self.constants[key] = constant
    
    def get_constant(self, key): 
        return self.constants[key]
    
    def add_list(self,key,list_object) :
        self.lists[key] = list_object
    
    def get_list(self,key) :
        return self.lists[key]
    
    def add_curve(self,key,curve) :
        self.curves[key] = curve
        
    def get_curve(self,key) :
        return self.curves[key]
    
    def add_environment(self, env) :
        #존재하는 값이 있으면 갱신
        for key in env.constants:
            self.constants[key] = env.constants[key]
        for key in env.lists :
            self.lists[key] = env.lists[key]
        for key in env.curves:
            self.curves[key]= env.curves[key]

In [15]:
#특별한 사항 없지만 instant 사용하는 간단한 예제
me_1 = market_environment('me_1',dt.datetime(2015,1,1))
me_1.add_list('symbols',['AAPL','MSFT','FB'])
me_1.get_list('symbols')

['AAPL', 'MSFT', 'FB']

In [17]:
me_2 = market_environment('me_2',dt.datetime(2015,1,1))
me_2.add_constant('volatility',0.2)
me_2.add_curve('short_rate',csr) # add instance of discounting class
me_2.get_curve('short_rate')

In [18]:
me_1.add_environment(me_2) #add complete environment

In [19]:
me_1.get_curve('short_rate')

In [20]:
me_1.constants, me_1.lists, me_1.curves

({'volatility': 0.2},
 {'symbols': ['AAPL', 'MSFT', 'FB']},
 {'short_rate': <__main__.constant_short_rate at 0x11366efc160>})